In [1]:
import pandas as pd

In [2]:
# DEFINE LOS LÍMITES PARA CALCULAR EL AQI DEL DIOXIDO DE AZUFRE
breakpoints = [
        (0, 50, 0, 35),
        (51, 100, 36, 75),
        (101, 150, 76, 185),
        (151, 200, 186, 304),
        (201, 300, 305, 604),
        (301, 500, 605, 1004),
]

# FUNCION PARA CALCULAR EL AQI DEL DIOXIDO DE AZUFRE EN PPB
def calculate_so2_aqi(so2_ppb):
    for I_lo, I_hi, C_lo, C_hi in breakpoints:
        if C_lo <= so2_ppb <= C_hi:
            aqi = ((I_hi - I_lo) / (C_hi - C_lo)) * (so2_ppb - C_lo) + I_lo
            return round(aqi)

    return None  # requires 1-hour O₃ AQI


In [3]:
# FUNCIÓN PARA ESTABLECER LA CATEGORÍA DE CADA AQI CALCULADO
def aqi_category(aqi):
    if aqi <= 50:
        return "Bueno"
    elif aqi <= 100:
        return "Moderado"
    elif aqi <= 150:
        return "No saludable para grupos sensibles"
    elif aqi <= 200:
        return "No saludable"
    elif aqi <= 300:
        return "Muy poco saludable"
    else:
        return "Peligroso"


In [4]:
# FUNCIÓN PARA PROCESAR LOS DATOS DEL DIOXIDO DE AZUFRE
def procesar_so2():
    # LEER EL EXCEL O3_Cotocollao.xlsx
    df_so2 = pd.read_excel("datos_entrenamiento/SO2_Cotocollao.xlsx")

    #ELIMINAR LAS FILAS CON VALORES NAN
    df_so2 = df_so2.dropna(subset=["COTOCOLLAO"])

    # CONVERTIR UG/M3 A PPB
    MW_SO2= 64.066
    df_so2.loc[:,"SO2_ppb"] = df_so2.loc[:,"COTOCOLLAO"] * (24.45 / MW_SO2)

    # CALCULAR EL AQI PARA CADA VALOR DE MUESTRA
    df_so2.loc[:,"SO2_AQI"] = df_so2.loc[:,"SO2_ppb"].apply(calculate_so2_aqi).round(0)

    # ESTABLECER UNA CATEGORIA PARA CADA AQI CALCULADO E IMPRIME EL DATASET PROCESADO
    df_so2.loc[:,"Categoria_SO2"] = df_so2.loc[:,"SO2_AQI"].apply(aqi_category)

    # EXPORTAR E IMPRIMIR EL DATASET PRCOESADO
    df_so2.to_excel("datos_procesados/datosProcesadosSO2.xlsx", index=False)
    print("SE HA PROCESADO EL DATASET CORRECTAMENTE!!")
    print(df_so2)

In [5]:
procesar_so2()

SE HA PROCESADO EL DATASET CORRECTAMENTE!!
                     Fecha  COTOCOLLAO   SO2_ppb  SO2_AQI Categoria_SO2
11439  2005-04-22 04:00:00    4.090385  1.561045        2         Bueno
11440  2005-04-22 05:00:00    4.090385  1.561045        2         Bueno
11441  2005-04-22 06:00:00    4.482692  1.710764        2         Bueno
11442  2005-04-22 07:00:00    4.482692  1.710764        2         Bueno
11443  2005-04-22 08:00:00    4.678846  1.785624        3         Bueno
...                    ...         ...       ...      ...           ...
191911 2025-11-30 19:00:00    0.240000  0.091593        0         Bueno
191912 2025-11-30 20:00:00    0.350000  0.133573        0         Bueno
191913 2025-11-30 21:00:00    0.850000  0.324392        0         Bueno
191914 2025-11-30 22:00:00    0.760000  0.290045        0         Bueno
191915 2025-11-30 23:00:00    0.500000  0.190819        0         Bueno

[170752 rows x 5 columns]
